## Update a Hosted Table

## AOIs: Update conservation concern on species lookup tables.
We have a lookup table per taxa (reptiles, amphibians, birds and mammlas) with a column **conservation concern** of each species, and we have a separate lookup table with a column **has_image**. We need both columns to be on the same table

In [45]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

In [46]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [ ]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [ ]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [ ]:
## Tables with conservation concern (cc)
cc_keys = {'amphibians':'eb487fb505e34052b4cb9e02f3f7a22c',
       'reptiles':'38356d976d3f43d7a0d2ab91034b054b',
       'mammals':'f6e7514c775442b39274d306b54a5952',
       'birds':'71e61cd2211b4670a28bfb14b3693f66'}
## Tables with has_image (hi)
hi_keys = {'amphibians':'a641a4cd269345dea93b8bcb1cb66676',
       'reptiles':'81c72a2a5ee6413699960b4c4bd9540f',
       'mammals':'84d3c71caf97479d85f620a4ee217d68',
       'birds':'4d8698734b654bb9bb7a61d9af314c76'}

In [ ]:
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [ ]:
## amphibians
cc = getHTfromId(cc_keys['amphibians'])
hi = getHTfromId(hi_keys['amphibians'])

In [ ]:
cc.head()

In [ ]:
hi.head()

In [ ]:
### Add conservation concern to hi table
hi = hi.merge(cc[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')

In [ ]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [ ]:
item = gis.content.get(hi_keys['amphibians'])
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, hi)

In [ ]:
fields_to_be_added

In [ ]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

In [ ]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [ ]:
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

In [ ]:
flayer.edit_features(updates= features_for_update)

In [ ]:
## Tables with conservation concern (cc)
cc_keys = {'birds':'71e61cd2211b4670a28bfb14b3693f66'}
## Tables with has_image (hi)
hi_keys = {'birds':'4d8698734b654bb9bb7a61d9af314c76'}

In [ ]:
### reptiles
hi_keys = {'reptiles':'81c72a2a5ee6413699960b4c4bd9540f'}
cc_keys = {'reptiles':'38356d976d3f43d7a0d2ab91034b054b'}

In [ ]:
for key in cc_keys:
    ## get df from Hosted tables
    cc = getHTfromId(cc_keys[key])
    hi = getHTfromId(hi_keys[key])
    
    ### bring conservation_concern column to base table
    hi = hi.merge(cc[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
    
    ### Create New fields
    item = gis.content.get(hi_keys[key])
    flayer = item.tables[0]
    fields_to_be_added = createFieldsToBeAdded(flayer, hi)
    
    ### Add new fields
    flayer.manager.add_to_definition({'fields':fields_to_be_added})
    
    ### Create features to update
    #### triplicate columns, check what has happened?
    if key == 'birds':
        start = np.arange(0,len(hi), step = 2000)
        for i in start:
            features_for_update = createFeaturesForUpdate(flayer = flayer, 
                                                          csv_table = hi[i:i+2000], 
                                                          fields_to_be_added =  fields_to_be_added, 
                                                          id_field_in_csv = "SliceNumber", 
                                                          id_field_in_service = "SliceNumber")
            flayer.edit_features(updates= features_for_update)
    else: 
        features_for_update = createFeaturesForUpdate(flayer = flayer,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")
    
        ### Update Features
        flayer.edit_features(updates= features_for_update)
    

## Add Conservation Concern field to amphibians table 

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

In [7]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [8]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [9]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [ ]:
## Function to get table from AGOL as sdf
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [ ]:
## Get sdf of amphibians tables
cc_key = 'eb487fb505e34052b4cb9e02f3f7a22c' # table with conservation concern
hi_key = 'a641a4cd269345dea93b8bcb1cb66676' # table used by FE

cc = getHTfromId(cc_key) 
hi = getHTfromId(hi_key) 


In [ ]:
cc.head(5)

In [ ]:
hi.head(5)

In [ ]:
### Add conservation concern, common name and synonyms to hi table 
hi = hi.merge(cc[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
hi.head()

In [ ]:
sum(hi['conservation_concern']==0)

In [ ]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [ ]:
# Get table to be updated and compare both to identify fields to be added
item = gis.content.get(hi_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, hi)

In [ ]:
fields_to_be_added

In [ ]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

In [ ]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [ ]:
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

In [ ]:
flayer.edit_features(updates= features_for_update)

## Add common name and synonyms fields
For some reason, when uploading more than one new field at a time, the resulting table in AGOL is incomplete. Therefore, in this case upload the two new fields separately
### Add common name field

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

#### Prepare table with common name

In [3]:
# Read table with common names shared by Scott (https://eowilson.maps.arcgis.com/home/item.html?id=b09984e429814a0ea7ea5ed44dd3b609)
cn = pd.read_csv('/Users/sofia/Documents/HE_Data/Amphibian_CRF_species_table.csv') 

In [4]:
cn.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,common_name,synonyms,Unnamed: 11
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,NaN,NaN,NaN
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,NaN,NaN,NaN
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,1,Northern Cricket Frog,NaN,NaN
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,1,"Southern Cricket Frog, Florida Cricket Frog (...",NaN,NaN
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,NaN,NaN,NaN


In [5]:
# Some values need to be modified
cn.loc[cn['Name'] == 'Leptodactylus_knudseni', 'common_name'].iloc[0]

"Knudsen's thin-toed frog, Rã-de-dedos-delgados-de-Knudsen, Rana de dedos delgados de Knudsen"

In [6]:
cn.loc[cn['Name'] == 'Leptobrachium_ailaonicum', 'synonyms'].iloc[0]

'Vibrissaphora echinata'

In [ ]:
# Modify values
# import re

# cn['common_name_array'] = np.nan

# for row in range(0,len(cn)):
#     if type(cn['common_name'][row]) == str:
#         cn['common_name_array'][row] = cn['common_name'][row].split(',')
#         l = len(cn['common_name_array'][row])
#         if l>1:
#             for i in range(0, l):
#                 a = re.sub(r"\([^()]*\)", "", ((cn['common_name_array'][row])[i]))
#                 a = a.rstrip().lstrip()
#                 cn['common_name_array'][row][i] = a
        

In [7]:
## Function that removes parenthesis (and anything inside them), and leading and trailing spaces
import re
def make_array(row):
    row_list = row.split(',')
    row_array= []
    for i in row_list: 
        row_array.append(re.sub(r'\([^()]*\)', '', i).rstrip().lstrip())
    return row_array

In [8]:
## Apply function to all rows in table and create an array of names
cn['common_name_array']= np.nan
cn['common_name_array'] = cn['common_name'].apply(lambda row : json.dumps(make_array(row), ensure_ascii=False) if type(row)==str else row)
cn['synonyms_array']= np.nan
cn['synonyms_array']= cn['synonyms'].apply(lambda row : json.dumps(make_array(row), ensure_ascii=False) if type(row)==str else row)

In [9]:
cn.loc[cn['Name'] == 'Leptodactylus_knudseni', 'common_name_array'].iloc[0]

'["Knudsen\'s thin-toed frog", "Rã-de-dedos-delgados-de-Knudsen", "Rana de dedos delgados de Knudsen"]'

In [10]:
cn.loc[cn['Name'] == 'Leptobrachium_ailaonicum', 'synonyms_array'].iloc[0]

'["Vibrissaphora echinata"]'

#### Update amphibians table with common names

In [9]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [10]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [11]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


In [12]:
## Function to get table from AGOL as sdf
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [17]:
# Call hosted table to update
hi_key = 'a641a4cd269345dea93b8bcb1cb66676' 
hi = getHTfromId(hi_key)

In [18]:
hi.head(5)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId,conservation_concern
0,Anodonthyla_hutchisoni,351,Anodonthyla hutchisoni,374,343,92,100,0,0,1,8
1,Amolops_wuyiensis,301,Amolops wuyiensis,75331,0,0,33,0,0,2,33
2,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,1,3,0
3,Allobates_fuscellus,101,Allobates fuscellus,564464,304538,54,15,0,0,4,0
4,Alytes_cisternasii,151,Alytes cisternasii,149939,48900,33,23,0,1,5,0


In [19]:
# Merge both tables to create new field in hosted table
hi = hi.merge(cn[['SliceNumber','common_name_array']],how='inner',on='SliceNumber')


In [20]:
hi.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId,conservation_concern,common_name_array
0,Anodonthyla_hutchisoni,351,Anodonthyla hutchisoni,374,343,92,100,0,0,1,8,NaN
1,Amolops_wuyiensis,301,Amolops wuyiensis,75331,0,0,33,0,0,2,33,NaN
2,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,1,3,0,NaN
3,Allobates_fuscellus,101,Allobates fuscellus,564464,304538,54,15,0,0,4,0,NaN
4,Alytes_cisternasii,151,Alytes cisternasii,149939,48900,33,23,0,1,5,0,"[""Iberian midwife toad"", ""Ibersiche geburtshel..."


In [21]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [22]:
# Get table to be updated and compare both to identify fields to be added
item = gis.content.get(hi_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, hi)

In [23]:
# Check fields to be added
fields_to_be_added

[{'name': 'common_name_array',
  'type': 'esriFieldTypeString',
  'actualType': 'nvarchar',
  'alias': 'common_name_array',
  'sqlType': 'sqlTypeOther',
  'length': 45000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [24]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [25]:
# Function to create features for update
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()
            

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)
            

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [27]:
# Create features for update
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

351 not available in service
301 not available in service
201 not available in service
101 not available in service
1 not available in service
352 not available in service
51 not available in service
251 not available in service
102 not available in service
202 not available in service
2 not available in service
353 not available in service
252 not available in service
103 not available in service
203 not available in service
153 not available in service
354 not available in service
53 not available in service
253 not available in service
104 not available in service
204 not available in service
355 not available in service
54 not available in service
254 not available in service
105 not available in service
205 not available in service
356 not available in service
55 not available in service
255 not available in service
5 not available in service
106 not available in service
156 not available in service
357 not available in service
56 not available in service
256 not available in serv

In [28]:
# Update table
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 5,
   'uniqueId': 5,
   'globalId': None,
   'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 15, 'uniqueId': 15, 'globalId': None, 'success': True},
  {'objectId': 17, 'uniqueId': 17, 'globalId': None, 'success': True},
  {'objectId': 22, 'uniqueId': 22, 'globalId': None, 'success': True},
  {'objectId': 23, 'uniqueId': 23, 'globalId': None, 'success': True},
  {'objectId': 29, 'uniqueId': 29, 'globalId': None, 'success': True},
  {'objectId': 30, 'uniqueId': 30, 'globalId': None, 'success': True},
  {'objectId': 31, 'uniqueId': 31, 'globalId': None, 'success': True},
  {'objectId': 37, 'uniqueId': 37, 'globalId': None, 'success': True},
  {'objectId': 38, 'uniqueId': 38, 'globalId': None, 'success': True},
  {'objectId': 44, 'uniqueId': 44, 'globalId': None, 'success': True},
  {'objectId': 45, 'uniqueId': 45, '

#### Add synonyms_array field

In [11]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [12]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [13]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

Keyring backend being used (keyring.backends.OS_X.Keyring (priority: 5)) either failed to install or is not recommended by the keyring project (i.e. it is not secure). This means you can not use stored passwords through GIS's persistent profiles. Note that extra system-wide steps must be taken on a Linux machine to use the python keyring module securely. Read more about this at the keyring API doc (http://bit.ly/2EWDP7B) and the ArcGIS API for Python doc (http://bit.ly/2CK2wG8).


In [14]:
## Function to get table from AGOL as sdf
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [50]:
# Call hosted table to update
hi_key = 'a641a4cd269345dea93b8bcb1cb66676' 
hi = getHTfromId(hi_key)

In [17]:
hi.head(5)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId,conservation_concern,common_name_array
0,Anodonthyla_hutchisoni,351,Anodonthyla hutchisoni,374,343,92,100,0,0,1,8,None
1,Amolops_wuyiensis,301,Amolops wuyiensis,75331,0,0,33,0,0,2,33,None
2,Ameerega_picta,201,Ameerega picta,1374742,609286,44,15,0,1,3,0,None
3,Allobates_fuscellus,101,Allobates fuscellus,564464,304538,54,15,0,0,4,0,None
4,Alytes_cisternasii,151,Alytes cisternasii,149939,48900,33,23,0,1,5,0,"[""Iberian midwife toad"", ""Ibersiche geburtshel..."


In [35]:
cn.loc[cn['Name'] == 'Rana_coreana', 'synonyms_array'].iloc[0]

'["Rana kunyuensis"]'

In [51]:
# Merge both tables to create new field in hosted table
hi = hi.merge(cn[['SliceNumber','synonyms_array']],how='right',on='SliceNumber')


In [52]:
hi.loc[cn['Name'] == 'Rana_coreana', 'synonyms_array'].iloc[0]

'["Rana kunyuensis"]'

In [46]:
len(hi)

6207

In [47]:
hi.head()

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,has_image,ObjectId,conservation_concern,common_name_array,synonyms_array
0,Acanthixalus_sonjae,1,Acanthixalus sonjae,787,787,100,100,0,0,6,0,None,NaN
1,Acanthixalus_spinosus,2,Acanthixalus spinosus,1041435,207623,20,15,0,0,14,0,None,NaN
2,Acris_crepitans,3,Acris crepitans,1750647,90682,5,15,0,1,22,10,"[""Northern Cricket Frog""]",NaN
3,Acris_gryllus,4,Acris gryllus,494322,24443,5,15,0,1,30,10,"[""Southern Cricket Frog"", ""Florida Cricket Fr...",NaN
4,Adelastes_hylonomos,5,Adelastes hylonomos,155,155,100,100,0,0,42,0,None,NaN


In [53]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [54]:
# Get table to be updated and compare both to identify fields to be added
item = gis.content.get(hi_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, hi)

In [55]:
# Check fields to be added
fields_to_be_added

[{'name': 'synonyms_array',
  'type': 'esriFieldTypeString',
  'actualType': 'nvarchar',
  'alias': 'synonyms_array',
  'sqlType': 'sqlTypeOther',
  'length': 45000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [56]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [57]:
# Function to create features for update
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()
            

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)
            

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [58]:
# Create features for update
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = hi,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

1 not available in service
2 not available in service
3 not available in service
4 not available in service
5 not available in service
6 not available in service
7 not available in service
8 not available in service
9 not available in service
10 not available in service
11 not available in service
12 not available in service
13 not available in service
14 not available in service
15 not available in service
16 not available in service
17 not available in service
18 not available in service
19 not available in service
20 not available in service
21 not available in service
22 not available in service
23 not available in service
24 not available in service
25 not available in service
26 not available in service
27 not available in service
28 not available in service
29 not available in service
30 not available in service
31 not available in service
32 not available in service
33 not available in service
34 not available in service
35 not available in service
36 not available in service
3

In [59]:
# Update table
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 264,
   'uniqueId': 264,
   'globalId': None,
   'success': True},
  {'objectId': 31, 'uniqueId': 31, 'globalId': None, 'success': True},
  {'objectId': 434, 'uniqueId': 434, 'globalId': None, 'success': True},
  {'objectId': 1023, 'uniqueId': 1023, 'globalId': None, 'success': True},
  {'objectId': 832, 'uniqueId': 832, 'globalId': None, 'success': True},
  {'objectId': 874, 'uniqueId': 874, 'globalId': None, 'success': True},
  {'objectId': 1481, 'uniqueId': 1481, 'globalId': None, 'success': True},
  {'objectId': 1905, 'uniqueId': 1905, 'globalId': None, 'success': True},
  {'objectId': 2324, 'uniqueId': 2324, 'globalId': None, 'success': True},
  {'objectId': 2346, 'uniqueId': 2346, 'globalId': None, 'success': True},
  {'objectId': 2472, 'uniqueId': 2472, 'globalId': None, 'success': True},
  {'objectId': 2577, 'uniqueId': 2577, 'globalId': None, 'success': True},
  {'objectId': 3139, 'uniqueId': 3139, 'globalId': None, 'success':

## Add Conservation Concern field to other taxa (mammals, reptiles and birds) 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
import json
import pandas as pd
from arcgis.features import FeatureLayerCollection
from copy import deepcopy
import os

In [2]:
# env_path = "../../.env"
env_path = ".env"
with open(env_path) as f:
   env = {}
   for line in f:
       env_key, _val = line.split("=")
       env_value = _val.split("\n")[0]
       env[env_key] = env_value

In [3]:
aol_password = env['ARCGIS_GRETA_PASS']
aol_username = env['ARCGIS_GRETA_USER']

In [5]:
gis = GIS("https://eowilson.maps.arcgis.com", aol_username, aol_password, profile = "eowilson")

In [6]:
## Function to get table from AGOL as sdf
def getHTfromId(item_id):
    item = gis.content.get(item_id)
    flayer = item.tables[0]
    sdf = flayer.query().sdf
    return sdf

In [37]:
## Get sdf of hosted tables
rep_key = '81c72a2a5ee6413699960b4c4bd9540f' 
bir_key = '4d8698734b654bb9bb7a61d9af314c76' 
mam_key = '84d3c71caf97479d85f620a4ee217d68'
rep = getHTfromId(rep_key) 
bir = getHTfromId(bir_key) 
mam = getHTfromId(mam_key)

In [38]:
rep.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId
0,Acanthosaura capra,1868,1129,60,90,0,101,0,1


In [17]:
bir.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId
0,Abeillia abeillei,73403,11203,15,34,0,1,1,1


In [18]:
mam.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId
0,Arctocebus calabarensis,75769,12714,17,33,0,301,1,1


In [15]:
### Add tables with conservation concerns and common name
path = '/Users/sofia/Documents/HE_Data/Lookup_Tables'
cc_rep = pd.read_csv('{0}/Reptile_CRF_species_table.csv'.format(path))
cc_bir = pd.read_csv('{0}/Bird_CRF_species_table.csv'.format(path))
cc_mam = pd.read_csv('{0}/Mammal_CRF_species_table.csv'.format(path))

In [39]:
cc_rep.head(1)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name
0,Ablepharus_bivittatus,1,Ablepharus bivittatus,154913,12860,8,22,0,14,1,"Twin-striped Skink, Two-Streaked Snake-Eyed Sk..."


In [19]:
cc_bir[cc_bir['Name']=='Baeopogon_clamans']

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name
989,Baeopogon_clamans,990,Baeopogon clamans,282897,53451,19,15,0,0,0,"Sj stedt's Greenbul, Sjöstedt's Greenbul, Whit..."


In [20]:
cc_mam.head(1)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name
0,Abditomys_latidens,1,Abditomys latidens,2850,212,7,84,0,77,0,Luzon Broad-toothed Rat


In [21]:
rep.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern
0,Acanthosaura capra,1868,1129,60,90,0,101,0,1,30


### Add conservation concern field
For some reason, when uploading more than one new field at a time, the resulting table in AGOL is incomplete. Therefore, in this case upload the two new fields separately


In [40]:
rep = rep.merge(cc_rep[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
rep.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern
0,Acanthosaura capra,1868,1129,60,90,0,101,0,1,30


In [22]:
bir = bir.merge(cc_bir[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
bir.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern
0,Abeillia abeillei,73403,11203,15,34,0,1,1,1,19


In [23]:
mam = mam.merge(cc_mam[['SliceNumber','conservation_concern']],how='inner',on='SliceNumber')
mam.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern
0,Arctocebus calabarensis,75769,12714,17,33,0,301,1,1,16


In [24]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [41]:
# Get table to be updated and compare both to identify fields to be added
item = gis.content.get(rep_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, rep)

In [42]:
fields_to_be_added

[{'name': 'conservation_concern',
  'type': 'esriFieldTypeDouble',
  'actualType': 'nvarchar',
  'alias': 'conservation_concern',
  'sqlType': 'sqlTypeOther',
  'length': 4000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [43]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [34]:
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [44]:
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = rep,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

In [46]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 7, 'uniqueId': 7, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': Non

### Add common name


In [48]:
## Get sdf of hosted tables
rep_key = '81c72a2a5ee6413699960b4c4bd9540f' 
bir_key = '4d8698734b654bb9bb7a61d9af314c76' 
mam_key = '84d3c71caf97479d85f620a4ee217d68'
rep = getHTfromId(rep_key) 
bir = getHTfromId(bir_key) 
mam = getHTfromId(mam_key)

In [49]:
rep.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern
0,Acanthosaura capra,1868,1129,60,90,0,101,0,1,30


In [50]:
cc_rep.head(1)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name
0,Ablepharus_bivittatus,1,Ablepharus bivittatus,154913,12860,8,22,0,14,1,"Twin-striped Skink, Two-Streaked Snake-Eyed Sk..."


In [51]:
## Function that removes parenthesis (and anything inside them), and leading and trailing spaces
import re
def make_array(row):
    row_list = row.split(',')
    row_array= []
    for i in row_list: 
        row_array.append(re.sub(r'\([^()]*\)', '', i).rstrip().lstrip())
    return row_array

In [53]:
## Apply function to all rows in table and create an array of names
cc_rep['common_name_array']= np.nan
cc_rep['common_name_array'] = cc_rep['common_name'].apply(lambda row : json.dumps(make_array(row), ensure_ascii=False) if type(row)==str else row)
cc_bir['common_name_array']= np.nan
cc_bir['common_name_array'] = cc_bir['common_name'].apply(lambda row : json.dumps(make_array(row), ensure_ascii=False) if type(row)==str else row)
cc_mam['common_name_array']= np.nan
cc_mam['common_name_array'] = cc_mam['common_name'].apply(lambda row : json.dumps(make_array(row), ensure_ascii=False) if type(row)==str else row)

In [56]:
cc_mam.head(1)

,Name,SliceNumber,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,conservation_concern,has_image,common_name,common_name_array
0,Abditomys_latidens,1,Abditomys latidens,2850,212,7,84,0,77,0,Luzon Broad-toothed Rat,"[""Luzon Broad-toothed Rat""]"


#### Update hosted tables with common names

In [58]:

mam = mam.merge(cc_mam[['SliceNumber','common_name_array']],how='inner',on='SliceNumber')
mam.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern,common_name_array
0,Arctocebus calabarensis,75769,12714,17,33,0,301,1,1,16.0,"[""Golden Angwantibo""]"


In [75]:
# rep = rep.merge(cc_rep[['SliceNumber','common_name_array']],how='inner',on='SliceNumber')
rep.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern,common_name_array
0,Acanthosaura capra,1868,1129,60,90,0,101,0,1,30,"[""Green Pricklenape"", ""Grüner Nackenstachler""]"


In [60]:
bir = bir.merge(cc_bir[['SliceNumber','common_name_array']],how='inner',on='SliceNumber')
bir.head(1)

,scientific_name,range_area_km2,wdpa_km2,percent_protected,conservation_target,is_flagship,SliceNumber,has_image,ObjectId,conservation_concern,common_name_array
0,Abeillia abeillei,73403,11203,15,34,0,1,1,1,19,"[""Emerald-chinned Hummingbird""]"


In [76]:
### Add new field to Hosted service
## Create fields
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = sdf = flayer.query().sdf
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            current_field['length'] = 45000
        if dt == 'int64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [89]:
# Get table to be updated and compare both to identify fields to be added
item = gis.content.get(bir_key)
flayer = item.tables[0]
fields_to_be_added = createFieldsToBeAdded(flayer, bir)

In [90]:
# Check fields to be added
fields_to_be_added

[{'name': 'common_name_array',
  'type': 'esriFieldTypeString',
  'actualType': 'nvarchar',
  'alias': 'common_name_array',
  'sqlType': 'sqlTypeOther',
  'length': 45000,
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [91]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [70]:
# Function to create features for update
#https://developers.arcgis.com/python/sample-notebooks/updating-features-in-a-feature-layer/
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()
            

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)
            

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [92]:
# Create features for update
features_for_update = createFeaturesForUpdate(flayer = flayer ,
                        csv_table = bir,
                        fields_to_be_added =  fields_to_be_added, 
                        id_field_in_csv = "SliceNumber", 
                        id_field_in_service = "SliceNumber")

7 not available in service
74 not available in service
75 not available in service
76 not available in service
77 not available in service
78 not available in service
79 not available in service
81 not available in service
82 not available in service
83 not available in service
84 not available in service
94 not available in service
224 not available in service
233 not available in service
234 not available in service
235 not available in service
236 not available in service
237 not available in service
238 not available in service
240 not available in service
242 not available in service
244 not available in service
245 not available in service
246 not available in service
256 not available in service
258 not available in service
260 not available in service
261 not available in service
264 not available in service
266 not available in service
269 not available in service
270 not available in service
271 not available in service
273 not available in service
274 not available in servic

In [93]:
# Update table
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 1,
   'uniqueId': 1,
   'globalId': None,
   'success': True},
  {'objectId': 2, 'uniqueId': 2, 'globalId': None, 'success': True},
  {'objectId': 3, 'uniqueId': 3, 'globalId': None, 'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True},
  {'objectId': 5, 'uniqueId': 5, 'globalId': None, 'success': True},
  {'objectId': 6, 'uniqueId': 6, 'globalId': None, 'success': True},
  {'objectId': 8, 'uniqueId': 8, 'globalId': None, 'success': True},
  {'objectId': 9, 'uniqueId': 9, 'globalId': None, 'success': True},
  {'objectId': 10, 'uniqueId': 10, 'globalId': None, 'success': True},
  {'objectId': 11, 'uniqueId': 11, 'globalId': None, 'success': True},
  {'objectId': 12, 'uniqueId': 12, 'globalId': None, 'success': True},
  {'objectId': 13, 'uniqueId': 13, 'globalId': None, 'success': True},
  {'objectId': 14, 'uniqueId': 14, 'globalId': None, 'success': True},
  {'objectId': 15, 'uniqueId': 15, 'globalId': N